In [1]:
import pandas as pd
from math import sqrt
import numpy as np

In [2]:
#Se carga dataset de peliculas y de ratings
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

In [3]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [6]:
#Se agrega el "usuario activo" a quién le vamos a recomendar películas de acuerdo a las que ha visto y calificado.

userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [9]:
df_ratings.userId.unique()

array([     1,      2,      3, ..., 247751, 247752, 247753])

In [30]:
pelis = [1968,1,2,296,1274]

In [34]:
for i in df_ratings.userId.unique():
    print(i)
    break

1


In [35]:
#Forma 1 de obtener los usuarios por movieId
lista_usuarios=[]
for i in df_ratings.userId.unique():   
    data = df_ratings[df_ratings.userId==i]
    if data[data["movieId"].isin(pelis)].shape[0]==5:
        lista_usuarios.append(i)
len(lista_usuarios)

KeyboardInterrupt: 

In [39]:
#Forma 2 de obtener los usuarios por movieId
data2 = df_ratings[df_ratings["movieId"].isin(pelis)]
data2

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
...,...,...,...
22883679,247738,296,4.0
22884132,247751,1,4.0
22884142,247751,296,4.0
22884164,247751,1274,5.0


In [63]:
#Forma 3 de obtener los usuarios por movieId
df10 = data2.groupby("userId", as_index=False).size()
usuariossimilares = df10[df10["size"]==5]["userId"].values

In [65]:
len(usuariossimilares)

1417

In [67]:
#Cogemos los primeros 50 usuarios 
usuariossimilares[:50]

array([  75,  106,  686,  815, 1040, 1130, 1502, 1599, 1625, 1950, 2065,
       2128, 2432, 2791, 2839, 2948, 3025, 3040, 3186, 3271, 3429, 3734,
       4099, 4208, 4282, 4292, 4415, 4586, 4725, 4818, 5104, 5165, 5547,
       6082, 6207, 6366, 6482, 6530, 7235, 7403, 7641, 7996, 8008, 8086,
       8245, 8572, 8675, 9101, 9358, 9663])

In [75]:
#valores de rating de inputMovies
inputMovies["rating"].values

array([3.5, 2. , 5. , 4.5, 5. ])

In [80]:
#Valores de las puntuaciones de las 5 peliculas para el usuario 815
data2[data2.userId==815]["rating"].values

array([4.5, 3. , 5. , 3. , 4.5])

In [83]:
# Creaccion de lista de Similitudes (Simility Matrix)
simi=[]
for i in usuariossimilares[:50]:
    vect1 = inputMovies["rating"].values
    vect2 = data2[data2.userId==i]["rating"].values
    simi.append(corr(vect1,vect2))
simi

[0.8272781516947568,
 0.5860090386731194,
 0.8320502943378437,
 0.5765566601970551,
 0.9434563530497265,
 0.2891574659831201,
 0.8770580193070292,
 0.43852900965351466,
 0.7161148740394327,
 0.1790287185098582,
 0.43852900965351466,
 0.5860090386731194,
 0.1386750490563073,
 0.8770580193070292,
 0.8204126541423671,
 -0.11720180773462387,
 0.45124262819714017,
 0.895143592549291,
 0.6784622064861937,
 0.2698959481797066,
 0.0,
 -0.1504142093990467,
 0.05860090386731193,
 0.29417420270727607,
 -0.43852900965351466,
 0.6564386345361467,
 -0.11183835382312353,
 -0.9024852563942803,
 -0.08006407690254358,
 0.4885967564883422,
 0.7674257668936506,
 -0.43852900965351466,
 0.1720052290384454,
 -0.047287799241095906,
 0.9615384615384617,
 0.6577935144802721,
 0.0,
 -0.3516054232038716,
 0.6981407669689395,
 0.11720180773462387,
 0.716114874039433,
 0.6266005147845038,
 -0.22562131409857009,
 0.6933752452815365,
 0.0,
 0.860026145192227,
 0.5370861555295746,
 -0.0860026145192227,
 0.692178738358

In [84]:
usuarios =usuariossimilares[:50] 
print(usuarios)

[  75  106  686  815 1040 1130 1502 1599 1625 1950 2065 2128 2432 2791
 2839 2948 3025 3040 3186 3271 3429 3734 4099 4208 4282 4292 4415 4586
 4725 4818 5104 5165 5547 6082 6207 6366 6482 6530 7235 7403 7641 7996
 8008 8086 8245 8572 8675 9101 9358 9663]


In [86]:
df_ratings[df_ratings["userId"]==75]

,userId,movieId,rating
7507,75,1,5.0
7508,75,2,3.5
7509,75,16,3.0
7510,75,19,2.5
7511,75,21,4.5
...,...,...,...
7872,75,30825,1.0
7873,75,30883,2.5
7874,75,33493,4.0
7875,75,33539,4.0


In [87]:
df20 = df_ratings[df_ratings["userId"].isin(usuarios)]

In [91]:
df30  = df20[~df20["movieId"].isin(pelis)]
df30

,userId,movieId,rating
7509,75,16,3.0
7510,75,19,2.5
7511,75,21,4.5
7512,75,25,4.5
7513,75,29,4.5
...,...,...,...
887476,9663,58559,4.5
887477,9663,59315,4.0
887478,9663,59615,3.0
887479,9663,68157,3.5


In [93]:
dfsimi = pd.DataFrame()
dfsimi["userId"]=usuarios
dfsimi["simi"] = simi
dfsimi

,userId,simi
0,75,0.827278
1,106,0.586009
2,686,0.832050
3,815,0.576557
4,1040,0.943456
5,1130,0.289157
6,1502,0.877058
7,1599,0.438529
8,1625,0.716115
9,1950,0.179029


In [96]:
df40  = pd.merge(df30, dfsimi, on= "userId")
df40

,userId,movieId,rating,simi
0,75,16,3.0,0.827278
1,75,19,2.5,0.827278
2,75,21,4.5,0.827278
3,75,25,4.5,0.827278
4,75,29,4.5,0.827278
...,...,...,...,...
53969,9663,58559,4.5,0.193973
53970,9663,59315,4.0,0.193973
53971,9663,59615,3.0,0.193973
53972,9663,68157,3.5,0.193973


In [97]:
df40["multi"] = df40["rating"]*df40["simi"]
df40

,userId,movieId,rating,simi,multi
0,75,16,3.0,0.827278,2.481834
1,75,19,2.5,0.827278,2.068195
2,75,21,4.5,0.827278,3.722752
3,75,25,4.5,0.827278,3.722752
4,75,29,4.5,0.827278,3.722752
...,...,...,...,...,...
53969,9663,58559,4.5,0.193973,0.872877
53970,9663,59315,4.0,0.193973,0.775891
53971,9663,59615,3.0,0.193973,0.581918
53972,9663,68157,3.5,0.193973,0.678905


In [103]:
df50 = df40.groupby("movieId",as_index=False)["simi"].sum()
df50

,movieId,simi
0,3,3.267725
1,4,-0.475183
2,5,5.974640
3,6,9.603589
4,7,2.742371
...,...,...
9440,149739,0.289157
9441,150270,0.294174
9442,150776,0.877058
9443,150780,0.877058


In [104]:
df60 = df40.groupby("movieId",as_index=False)["multi"].sum()
df60

,movieId,multi
0,3,8.726945
1,4,-1.388421
2,5,13.115650
3,6,32.243301
4,7,8.659259
...,...,...
9440,149739,0.578315
9441,150270,0.441261
9442,150776,2.631174
9443,150780,3.508232


In [106]:
df70  = pd.merge(df50,df60,on="movieId")
df70["recom"]= df70["multi"]/df70["simi"]
df70

,movieId,simi,multi,recom
0,3,3.267725,8.726945,2.670649
1,4,-0.475183,-1.388421,2.921867
2,5,5.974640,13.115650,2.195220
3,6,9.603589,32.243301,3.357422
4,7,2.742371,8.659259,3.157581
...,...,...,...,...
9440,149739,0.289157,0.578315,2.000000
9441,150270,0.294174,0.441261,1.500000
9442,150776,0.877058,2.631174,3.000000
9443,150780,0.877058,3.508232,4.000000


In [110]:
ids= df70.sort_values("recom", ascending=False).head(10)["movieId"].values

In [112]:
df_movies[df_movies["movieId"].isin(ids)]

,movieId,title
933,950,"Thin Man, The"
5523,5621,"Tuxedo, The"
5607,5705,Xanadu
8292,8974,"SpongeBob SquarePants Movie, The"
10944,44709,Akeelah and the Bee
14295,71518,Whip It
18161,90717,Tower Heist
21405,103655,R.I.P.D.
21452,103810,Red 2
21754,104913,Rush


In [81]:
# Creo la función de correlación para encontrar la similitud 
# entre los usuarios 

def corr(v1,v):
    x=0
    y=0
    z=0
    for i in range(len(v)):
        x+=(v1[i]-v1.mean())*(v[i]-v.mean())
        y+=(v1[i]-v1.mean())**2
        z+=(v[i]-v.mean())**2
    y=np.sqrt(y)
    z=np.sqrt(z)
    return x/(y*z)
        

In [53]:
df10 = pd.DataFrame(data2.groupby("userId", as_index=False).size())
df10[df10["size"]==5]

,userId,size
34,75,5
45,106,5
323,686,5
388,815,5
505,1040,5
...,...,...
115568,246506,5
115583,246533,5
115784,246959,5
115811,246999,5


In [45]:
lista= []
for i in df_ratings.userId.unique():
    if data2[data2.userId == i].shape[0]==5:
        lista.append(i)

KeyboardInterrupt: 

In [ ]:
len(lista)

In [8]:
len(df_ratings["userId"].unique())

247753

# aqui

In [10]:
df_ratings[df_ratings["userId"]==1]

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435


In [5]:
#Mostramos que el usuario con el Id 100, ha visto y ranqueado 81 películas
df_ratings[df_ratings["userId"]==100]

,userId,movieId,rating,timestamp
8908,100,11,4.0,1047773238
8909,100,39,2.0,1047773211
8910,100,50,4.5,1180558130
8911,100,153,4.0,1180558106
8912,100,231,2.5,1180558151
...,...,...,...,...
8984,100,8360,4.5,1180557987
8985,100,33794,4.0,1180557952
8986,100,41569,3.5,1180558238
8987,100,45722,4.5,1180558208


In [16]:
# Se crea una función para eliminar el año de lanzamiento de los nombres de cada película en la columna "title"
def elim_fech(text):
    return text[:text.rfind("(")].strip() #se hace un slicing del inicio hasta el primer parentesis
# se busca de derecha a izq. utilizando .rfind() y el .strip() elimina los espacios al inicio final

In [17]:
#Se utiliza apply para eliminar el año de las películas 
df_movies["title"] = df_movies["title"].apply(elim_fech) 

In [18]:
#Vemos cuantos usuarios han ranqueado películas
len(df_ratings.userId.unique())

247753

In [19]:
#Eliminamos la columna "genres"
df_movies.drop("genres", axis=1, inplace=True)

In [20]:
#Eliminamos la columna "timestamp"

df_ratings.drop("timestamp", axis=1, inplace=True)

In [21]:
df_ratings.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [22]:
#Se agrega el "usuario activo" a quién le vamos a recomendar películas de acuerdo a las que ha visto y calificado.

userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [23]:
#Se hace un marge para al dataframe df_movies agregarle el Id de las películas
# y se obtiene un nuevo dataframe llamado inputMovies
Ids = df_movies[df_movies['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(Ids, inputMovies)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


## Buscar las peliculas en rating 

In [14]:
#Ids de las películas que ha visto y ranqueado el usuario activo
inputMovies['movieId'].tolist() 

[1, 2, 296, 1274, 1968]

In [15]:
#Del dataframe df_ratings obtener todas las veces que se han ranqueado las peliculas que ha visto el usuario activo
userSubset = df_ratings[df_ratings['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [16]:
# Muestro las películas que ha ranquedo el usuario 75
userSubset[userSubset["userId"]==75]

,userId,movieId,rating
7507,75,1,5.0
7508,75,2,3.5
7540,75,296,5.0
7633,75,1274,4.5
7673,75,1968,5.0


In [17]:
# Si el usuario ha ranqueado 5 peliculas guardo su Id en Ids
# con el objetivo de obtener los usuarios que han ranqueado las mismas
#películas que el usuario activo

ids = []
for i in userSubset["userId"].unique():
    if userSubset[userSubset["userId"]==i].shape[0] > 4:
        ids.append(i)

In [18]:
# Para saber cuantos usuarios coinciden con nuestro usuario activo
len(ids)

1417

In [19]:
# Creo la función de correlación para encontrar la similitud 
# entre los usuarios 

def corr(v1,v):
    x=0
    y=0
    z=0
    for i in range(len(v)):
        x+=(v1[i]-v1.mean())*(v[i]-v.mean())
        y+=(v1[i]-v1.mean())**2
        z+=(v[i]-v.mean())**2
    y=np.sqrt(y)
    z=np.sqrt(z)
    return x/(y*z)
        

In [20]:
# de los 1417 usuarios que han visto la misma película que el 
# usuario activo, solo nos quedamos con los primeros 50
ids2 = ids[:50]
len(ids2)

50

In [21]:
# v es el vector con la info de las calificaciones que ha dado el usuario activo
# y se recorren los ids para guardar en la lista correlaciones
# la correlación entre el usuario activo y los dem
v = inputMovies["rating"].values
correlaciones=[]
for i in ids2:
    v1 = userSubset[userSubset["userId"]==i]["rating"].values
    correlaciones.append(corr(v,v1))

In [22]:
# Se crea el dataframe df_corr con el id y su correlación con el usuario activo
df_corr = pd.DataFrame()
df_corr["ids"] = ids2
df_corr["correlacion"] = correlaciones
df_corr.head()

,ids,correlacion
0,75,0.827278
1,106,0.586009
2,686,0.832050
3,815,0.576557
4,1040,0.943456


In [23]:
# Nos quedamos con los 10 usuarios con mayor correlación 
df_corr = df_corr.sort_values("correlacion",ascending=False).head(10)
df_corr

,ids,correlacion
34,6207,0.961538
4,1040,0.943456
17,3040,0.895144
6,1502,0.877058
13,2791,0.877058
45,8572,0.860026
2,686,0.832050
0,75,0.827278
14,2839,0.820413
30,5104,0.767426


In [24]:
# Se hace un merge entre el df_corr y el df_ratings 
# Como resultado se obtienen las películas que se van a recomendar
# con la infor del usuario que la vio, el rating y la correlación
topUsersRating=df_corr.merge(df_ratings, left_on='ids', right_on='userId', how='inner')
topUsersRating

,ids,correlacion,userId,movieId,rating
0,6207,0.961538,6207,1,3.5
1,6207,0.961538,6207,2,2.0
2,6207,0.961538,6207,5,1.5
3,6207,0.961538,6207,10,2.5
4,6207,0.961538,6207,16,3.5
...,...,...,...,...,...
6980,5104,0.767426,5104,109487,3.0
6981,5104,0.767426,5104,112852,3.0
6982,5104,0.767426,5104,115149,3.5
6983,5104,0.767426,5104,116797,3.0


In [25]:
#Se agrega la columna "WeightedRating" resultado de multiplicar la correlación con el rating 
topUsersRating['weightedRating'] = topUsersRating['correlacion']*topUsersRating['rating']
topUsersRating

,ids,correlacion,userId,movieId,rating,weightedRating
0,6207,0.961538,6207,1,3.5,3.365385
1,6207,0.961538,6207,2,2.0,1.923077
2,6207,0.961538,6207,5,1.5,1.442308
3,6207,0.961538,6207,10,2.5,2.403846
4,6207,0.961538,6207,16,3.5,3.365385
...,...,...,...,...,...,...
6980,5104,0.767426,5104,109487,3.0,2.302277
6981,5104,0.767426,5104,112852,3.0,2.302277
6982,5104,0.767426,5104,115149,3.5,2.685990
6983,5104,0.767426,5104,116797,3.0,2.302277


In [26]:
#Se aplica un groupby para agrupar por película sumando correlación y weightedRating
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['correlacion','weightedRating']]
tempTopUsersRating

,correlacion,weightedRating
movieId,,
1,8.661447,32.952520
2,8.661447,23.866380
3,1.587838,4.869489
5,1.728964,2.209733
6,2.459502,5.469594
...,...,...
148452,0.877058,2.631174
148454,0.877058,3.508232
149354,0.877058,2.631174


In [27]:
#Se crea un df vacio el cual va a contener la info de las películas a recomendar 
recommendation_df = pd.DataFrame()

In [28]:
#Se obtiene el "weighted average recommendation score"

recommendation_df['weighted average recommendation score'] = tempTopUsersRating['weightedRating']/tempTopUsersRating['correlacion']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.804505,1
2,2.755472,2
3,3.066741,3
5,1.278068,5
6,2.223862,6


In [29]:
#Se ordena por weighted average recommendation score
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
93988,5.0,93988
98981,5.0,98981
7042,5.0,7042
8493,5.0,8493
5294,5.0,5294
4381,5.0,4381
4467,5.0,4467
3468,5.0,3468
2289,5.0,2289


In [30]:
#Se obtienen las películas a recomendar

df_movies[df_movies["movieId"].isin(recommendation_df.index[:10].tolist())]

,movieId,title
277,280,Murder in the First
2205,2289,"Player, The"
3380,3468,"Hustler, The"
4287,4381,"Closet, The (Placard, Le)"
4373,4467,"Adventures of Baron Munchausen, The"
5198,5294,Frailty
6931,7042,Betty Blue (37°2 le matin)
7849,8493,Memphis Belle
18942,93988,North & South
20149,98981,"Arrival of a Train, The"
